In [34]:
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix,matthews_corrcoef,balanced_accuracy_score
import sys 
sys.path.append('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/')
from RF_atomver import *
import pandas as pd
import numpy as np
import pickle

In [35]:
result_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF_grid_search/rf_results/'
datapath1 = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK2/bind/'
testX = pd.read_csv(datapath1+'NEK2_binding_moe_UNDER_testX.csv',header=None).to_numpy()
testy = pd.read_csv(datapath1+'NEK2_binding_moe_UNDER_test_y.csv',header=None).to_numpy()
with open('NEK2_binding_moe_UNDER_RF_BCW_GS_gridsearch.pkl', 'rb') as f: 
    gs1 = pickle.load(f)
with open('NEK2_binding_moe_UNDER_RF_BCW_GS_best_model.pkl', 'rb') as f: 
    bestmodel = pickle.load(f)

In [36]:
testX.shape

(284, 306)

In [37]:
gs1_best_est = gs1.best_estimator_
results1 = rf_results(gs1_best_est, testX, testy)
pred1 = gs1_best_est.predict(testX)
cm1 = confusion_matrix(testy,pred1)
cm1

array([[207,  65],
       [  5,   7]])

In [38]:
bestmodel
results2 = rf_results(bestmodel, testX, testy)
pred2 = bestmodel.predict(testX)
cm2= confusion_matrix(testy,pred2)
cm2

array([[207,  65],
       [  5,   7]])

In [39]:
metric_cols = ['accuracy', 'precision', 'recall', 'specificity',
               'y', 'TN', 'FN', 'FP', 'TP',
       'model', 'cm', 'prediction_type', 'NEK', 'feat_type', 'strategy',
       'RF_type']

In [40]:
def add_cm(df): 
    # print(filepath)
    # print(filename)
    # df = pd.read_csv(filepath+filename)
    
    true_labels = df['y'] 
    predictions = df['prediction']
    cm = confusion_matrix(true_labels, predictions )
    cm_flattened = cm.flatten().tolist()
    df['cm'] = [cm_flattened]* len(df)
    df['prediction_type'] = df.apply(lambda x: prediction_type(x['y'], x['prediction']), axis=1)
    return df
        

In [41]:
def more_results(df, X, y,model_name, NEK, rf_type, feat_type, strategy, subset): 
    true_labels = df['y'] 
    predictions = df['prediction']
    cm = confusion_matrix(true_labels, predictions )
    cm_flattened = cm.flatten().tolist()
    df['model'] = model_name 
    df['NEK'] = NEK
    df['strategy'] = strategy
    df['RF_type'] = rf_type
    df['strategy'] = strategy 
    df['feat_type'] = feat_type
    df['dataset_category']=df['feat_type'].astype(str)+'_'+df['strategy']
    df['cm'] = [cm_flattened]* len(df)
    df['prediction_type'] = df.apply(lambda x: prediction_type(x['y'], x['prediction']), axis=1)
    df['ROC-AUC'] = roc_auc_score(df['y'], df['prediction'])
    
    df['MCC'] = matthews_corrcoef(df['y'], df['prediction'])
    df['Balanced Accuracy'] = balanced_accuracy_score(df['y'], df['prediction'])
    
    df['f1'] = f1_score(df['y'], df['prediction'])
    
    df['prediction_type'] = df.apply(lambda x: prediction_type(x['y'], x['prediction']), axis=1)
    df['subset'] = subset
    return df

In [42]:
neks = ['NEK2_binding', 'NEK2_inhibition', 'NEK3_binding', 'NEK5_binding', 'NEK9_binding', 'NEK9_inhibition'] 
nek_list = ["2", "3", "5", "9"]
nektype = ['binding','inhibition']
feat_types = ['moe', 'mfp']
samplings = ['scaled', 'UNDER' , 'SMOTE', 'ADASYN']
model_types = ['RF','RF_BCW', 'BRFC', 'BRFC_BCW']

count = 0 
rfs=[]
feats = [] 
samps = [] 
NEKS = []
models = [] 

for n in nek_list:
    for i in nektype: 
        for feat in feat_types: 
            for samp in samplings:  
                for rf in model_types:  # RF, RF_BCW, BRFC, BRFC_BCW
                    if i == 'binding':
                        data_path = f'/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK{n}/bind/'
                    
                        df_name = f'NEK{n}_{i}_{feat}_{samp}_df.csv'
                        count+=1
                        trainX, trainy, testX, testy = get_arrays(data_path, df_name)
                        model_name = f'NEK{n}_{i}_{feat}_{samp}_{rf}_GS_gridsearch'
                        print(f'{count}. {model_name}')
                        # models.append(model_name)
                        # rfs.append(rf)
                        # samps.append(samp)
                        # NEKS.append(f'NEK{n}_binding') 
                        # feats.append(feat) 
                        with open(f'{model_name}.pkl', 'rb') as f:
                            model = pickle.load(f)
                        train_df = gather_rf_results(model, trainX, trainy)
                        test_df = gather_rf_results(model, testX, testy)
                        model_nameshort = f'NEK{n}_{i}_{feat}_{samp}_{rf}_GS'
                        train_df = more_results(train_df, trainX, trainy, model_nameshort, f'NEK{n}_binding',  rf, feat, samp, 'train')
                        test_df = more_results(test_df, testX, testy, model_nameshort, f'NEK{n}_binding',  rf, feat, samp, 'test')
                        train_df.to_csv(f'{model_nameshort}_train.csv',index=False) 
                        test_df.to_csv(f'{model_nameshort}_test.csv',index=False) 
                        
                        
                        
                    if n in ['2', '9'] and i == 'inhibition': 
                        data_path = f'/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK{n}/inhib/'
                        df_name = f'NEK{n}_{i}_{feat}_{samp}_df.csv'
                        trainX, trainy, testX, testy = get_arrays(data_path, df_name)
    
                        count+=1
                        model_name = f'NEK{n}_{i}_{feat}_{samp}_{rf}_GS_gridsearch'
                        print(f'{count}. {model_name}')
                        with open(f'{model_name}.pkl', 'rb') as f:
                            model =pickle.load(f) 
                        model_nameshort = f'NEK{n}_{i}_{feat}_{samp}_{rf}_GS'
                        train_df = gather_rf_results(model, trainX, trainy)
                        test_df = gather_rf_results(model, testX, testy)
                        train_df = more_results(train_df, trainX, trainy, model_nameshort, f'NEK{n}_inhibition',  rf, feat, samp, 'train')
                        test_df = more_results(test_df, testX, testy,model_nameshort, f'NEK{n}_inhibition',  rf, feat, samp, 'test')
                        train_df.to_csv(f'{model_nameshort}_train.csv',index=False) 
                        test_df.to_csv(f'{model_nameshort}_test.csv',index=False) 
                        
                        

train X shape: (1125, 306), y: (1125,), test X: (283, 306), y:(283,)
1. NEK2_binding_moe_scaled_RF_GS_gridsearch
train X shape: (1125, 306), y: (1125,), test X: (283, 306), y:(283,)
2. NEK2_binding_moe_scaled_RF_BCW_GS_gridsearch


/Users/jayceepang/msse/capstone/atomsci/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jayceepang/msse/capstone/atomsci/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train X shape: (1125, 306), y: (1125,), test X: (283, 306), y:(283,)
3. NEK2_binding_moe_scaled_BRFC_GS_gridsearch
train X shape: (1125, 306), y: (1125,), test X: (283, 306), y:(283,)
4. NEK2_binding_moe_scaled_BRFC_BCW_GS_gridsearch
train X shape: (90, 306), y: (90,), test X: (283, 306), y:(283,)
5. NEK2_binding_moe_UNDER_RF_GS_gridsearch
train X shape: (90, 306), y: (90,), test X: (283, 306), y:(283,)
6. NEK2_binding_moe_UNDER_RF_BCW_GS_gridsearch
train X shape: (90, 306), y: (90,), test X: (283, 306), y:(283,)
7. NEK2_binding_moe_UNDER_BRFC_GS_gridsearch
train X shape: (90, 306), y: (90,), test X: (283, 306), y:(283,)
8. NEK2_binding_moe_UNDER_BRFC_BCW_GS_gridsearch
train X shape: (2160, 306), y: (2160,), test X: (283, 306), y:(283,)
9. NEK2_binding_moe_SMOTE_RF_GS_gridsearch
train X shape: (2160, 306), y: (2160,), test X: (283, 306), y:(283,)
10. NEK2_binding_moe_SMOTE_RF_BCW_GS_gridsearch
train X shape: (2160, 306), y: (2160,), test X: (283, 306), y:(283,)
11. NEK2_binding_moe_SMO

/Users/jayceepang/msse/capstone/atomsci/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jayceepang/msse/capstone/atomsci/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train X shape: (1122, 306), y: (1122,), test X: (282, 306), y:(282,)
67. NEK3_binding_moe_scaled_BRFC_GS_gridsearch
train X shape: (1122, 306), y: (1122,), test X: (282, 306), y:(282,)
68. NEK3_binding_moe_scaled_BRFC_BCW_GS_gridsearch
train X shape: (128, 306), y: (128,), test X: (282, 306), y:(282,)
69. NEK3_binding_moe_UNDER_RF_GS_gridsearch
train X shape: (128, 306), y: (128,), test X: (282, 306), y:(282,)
70. NEK3_binding_moe_UNDER_RF_BCW_GS_gridsearch
train X shape: (128, 306), y: (128,), test X: (282, 306), y:(282,)
71. NEK3_binding_moe_UNDER_BRFC_GS_gridsearch
train X shape: (128, 306), y: (128,), test X: (282, 306), y:(282,)
72. NEK3_binding_moe_UNDER_BRFC_BCW_GS_gridsearch
train X shape: (2116, 306), y: (2116,), test X: (282, 306), y:(282,)
73. NEK3_binding_moe_SMOTE_RF_GS_gridsearch
train X shape: (2116, 306), y: (2116,), test X: (282, 306), y:(282,)
74. NEK3_binding_moe_SMOTE_RF_BCW_GS_gridsearch
train X shape: (2116, 306), y: (2116,), test X: (282, 306), y:(282,)
75. NEK3_

/Users/jayceepang/msse/capstone/atomsci/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jayceepang/msse/capstone/atomsci/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train X shape: (1126, 306), y: (1126,), test X: (283, 306), y:(283,)
131. NEK9_binding_moe_scaled_BRFC_GS_gridsearch
train X shape: (1126, 306), y: (1126,), test X: (283, 306), y:(283,)
132. NEK9_binding_moe_scaled_BRFC_BCW_GS_gridsearch
train X shape: (96, 306), y: (96,), test X: (283, 306), y:(283,)
133. NEK9_binding_moe_UNDER_RF_GS_gridsearch
train X shape: (96, 306), y: (96,), test X: (283, 306), y:(283,)
134. NEK9_binding_moe_UNDER_RF_BCW_GS_gridsearch
train X shape: (96, 306), y: (96,), test X: (283, 306), y:(283,)
135. NEK9_binding_moe_UNDER_BRFC_GS_gridsearch
train X shape: (96, 306), y: (96,), test X: (283, 306), y:(283,)
136. NEK9_binding_moe_UNDER_BRFC_BCW_GS_gridsearch
train X shape: (2156, 306), y: (2156,), test X: (283, 306), y:(283,)
137. NEK9_binding_moe_SMOTE_RF_GS_gridsearch
train X shape: (2156, 306), y: (2156,), test X: (283, 306), y:(283,)
138. NEK9_binding_moe_SMOTE_RF_BCW_GS_gridsearch
train X shape: (2156, 306), y: (2156,), test X: (283, 306), y:(283,)
139. NEK9

In [43]:
test_df

,prediction,accuracy,precision,recall,specificity,prob_class0,prob_class1,y,TN,FN,...,RF_type,feat_type,dataset_category,cm,prediction_type,ROC-AUC,MCC,Balanced Accuracy,f1,subset
0,0,0.9375,0.833333,0.555556,0.985915,1.00,0.00,0,70,4,...,BRFC_BCW,mfp,mfp_ADASYN,"[70, 1, 4, 5]",TN,0.770736,0.649582,0.770736,0.666667,test
1,0,0.9375,0.833333,0.555556,0.985915,0.99,0.01,0,70,4,...,BRFC_BCW,mfp,mfp_ADASYN,"[70, 1, 4, 5]",TN,0.770736,0.649582,0.770736,0.666667,test
2,0,0.9375,0.833333,0.555556,0.985915,0.96,0.04,0,70,4,...,BRFC_BCW,mfp,mfp_ADASYN,"[70, 1, 4, 5]",TN,0.770736,0.649582,0.770736,0.666667,test
3,0,0.9375,0.833333,0.555556,0.985915,1.00,0.00,0,70,4,...,BRFC_BCW,mfp,mfp_ADASYN,"[70, 1, 4, 5]",TN,0.770736,0.649582,0.770736,0.666667,test
4,0,0.9375,0.833333,0.555556,0.985915,0.80,0.20,0,70,4,...,BRFC_BCW,mfp,mfp_ADASYN,"[70, 1, 4, 5]",TN,0.770736,0.649582,0.770736,0.666667,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1,0.9375,0.833333,0.555556,0.985915,0.03,0.97,1,70,4,...,BRFC_BCW,mfp,mfp_ADASYN,"[70, 1, 4, 5]",TP,0.770736,0.649582,0.770736,0.666667,test
76,0,0.9375,0.833333,0.555556,0.985915,0.96,0.04,1,70,4,...,BRFC_BCW,mfp,mfp_ADASYN,"[70, 1, 4, 5]",FN,0.770736,0.649582,0.770736,0.666667,test
77,1,0.9375,0.833333,0.555556,0.985915,0.02,0.98,1,70,4,...,BRFC_BCW,mfp,mfp_ADASYN,"[70, 1, 4, 5]",TP,0.770736,0.649582,0.770736,0.666667,test
78,0,0.9375,0.833333,0.555556,0.985915,0.64,0.36,1,70,4,...,BRFC_BCW,mfp,mfp_ADASYN,"[70, 1, 4, 5]",FN,0.770736,0.649582,0.770736,0.666667,test


In [44]:
test_df.columns

Index(['prediction', 'accuracy', 'precision', 'recall', 'specificity',
       'prob_class0', 'prob_class1', 'y', 'TN', 'FN', 'FP', 'TP', 'model',
       'NEK', 'strategy', 'RF_type', 'feat_type', 'dataset_category', 'cm',
       'prediction_type', 'ROC-AUC', 'MCC', 'Balanced Accuracy', 'f1',
       'subset'],
      dtype='object')